In [11]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import linalg

# generate 50 2D-data points
def actual_dis():
    x = np.arange(0,1,0.02)
    y = np.sin(x**2 + 1)
    return x,y

# add Guassian noise
def add_noise(y_origin):
    mu = 0
    sigma = 0.04
    noise = np.random.normal(mu, sigma, len(y_origin))  
    y_noise = y_origin + noise
    return y_noise

# fit the original data
def poly_fitting(x, y, M):
    z = np.polyfit(x, y, M)
    f = np.poly1d(z)
    return f

# generate original data
x_true,y_ture = actual_dis()

# fit on the original data
M1 = 3
f = poly_fitting(x_true, y_ture, M1)
#x_curve = np.linspace(x_true[0], x_true[-1],50)
x_curve = x_true
y_curve = f(x_curve)

# add_noise
y_noise = add_noise(y_curve)
x_noise = x_curve

def guassian_basis(x, mu, s=1):
    return np.exp(-s * linalg.norm(mu - x)**2)

def mle_basis_func(x, y, M, s):
    N = len(x)
    phi = np.zeros([N,M])
    vy = np.zeros([N,1])
    
    # calculate the guassian_basis matrix
    for m in range(M):
        mu = m/M
        phi[:, m] = np.vectorize(guassian_basis)(x, mu, s)
    
    for n in range(0, N):
        vy[n][0] = y[n]
        
    # mphi = np.dot(phi.T, phi)
    # imphi = linalg.inv(mphi)
    # tmp = np.dot(imphi, phi.T)
    
    tmp = linalg.pinv(phi)
    
    w = np.dot(tmp, vy)
    return w, phi

def mle_basis_plot(x,w):
    M = len(w)
    N = len(x)
    phi = np.zeros([N,M])
    
    for m in range(M):
        mu = m/M
        phi[:, m] = np.vectorize(guassian_basis(x, mu))
                
    y = np.dot(phi, w)
    return x, y

#plot
M2 = 100
s = 1
[w, phi_guas] = mle_basis_func(x_noise, y_noise, M2, s)
[x_map, y_map] = mle_basis_plot(x_curve, w)
# plot
plt.title("M = "+str(M2)+" s = "+str(s))
plt.plot(x_true, y_ture, 'ro',markevery=3)
plt.plot(x_curve, y_curve, 'red')
plt.plot(x_noise, y_noise, 'go')
plt.plot(x_map, y_map)
plt.show

TypeError: float() argument must be a string or a real number, not 'vectorize'